In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.metrics import jaccard_similarity_score

In [2]:
desc = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/2_1_2_removed_descriptor.tsv")

In [3]:
drug_mat = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/8_gold_standard_drug_binary_matrix.tsv", index_col=0)

In [16]:
#2017.3.15. drug feature를 independent feature 만큼만 제한하기
drug_feature = desc.columns.values[3:1850]
disease_feature = desc.columns.values[1850:]
print len(drug_feature), len(disease_feature)

1847 3521


In [19]:
# removed drug feature와 drug의 matrix
drug_mat2 = drug_mat.filter(items = drug_feature, axis=0)

In [20]:
#2017.3.15 drug-drug_similarity 구하기(removed)
gold_drug_id = drug_mat2.columns.values
df_dr_sim = pd.DataFrame()
mat_dr_sim = pd.DataFrame(columns = gold_drug_id, index = gold_drug_id)
cnt = 0
print 'start : ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
for i in gold_drug_id:
    desc1 = drug_mat2[i]
    for j in gold_drug_id:
        desc2 = drug_mat2[j]
        
        sim = jaccard_similarity_score(desc1, desc2)
        
        #dataframe형
        temp_df = pd.DataFrame(data= [[i, j , sim]], columns=['Drug_id1', 'Drug_id2', 'Similarity'])
        df_dr_sim = df_dr_sim.append(temp_df)
        # matrix형
        mat_dr_sim[i].ix[j] = sim
    cnt = cnt + 1
    if cnt % 100 == 0:
        print 'cnt == {} : '.format(cnt) + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print 'end : ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S')

start : 2017-03-15 15:37:19
cnt == 100 : 2017-03-15 15:38:44
cnt == 200 : 2017-03-15 15:40:30
cnt == 300 : 2017-03-15 15:42:39
cnt == 400 : 2017-03-15 15:45:11
end : 2017-03-15 15:47:13


In [22]:
df_dr_sim.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/10_2_gold_drug_drug_sim_removed.tsv", sep='\t', index=False)
df_dr_sim.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/10_2_gold_drug_drug_sim_removed.tsv", sep='\t', index=False)
mat_dr_sim.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/10_2_gold_drug_drug_sim_mat_removed.tsv", sep='\t')
mat_dr_sim.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/10_2_gold_drug_drug_sim_mat_removed.tsv", sep='\t')

In [21]:
# disease는 feature의 변화가 없으니 이전에 구해둔 것으로 해도 된다.

In [23]:
#2017.3.15. drug-disease distance #normalize 안함.
df_dr_sim = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/10_2_gold_drug_drug_sim_removed.tsv")
df_di_sim = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/6_new_disease/10_gold_disease_disease_sim.tsv")
df_asso = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/7_new_association/7_gold_association.tsv")

In [32]:
gold_drug_id = df_asso['drug_id'].drop_duplicates()
gold_disease_id = df_asso['disease_id'].drop_duplicates()
di_dr_mat = pd.DataFrame(columns = gold_drug_id, index= gold_disease_id) #di에 대한 dr의 distance보기
cnt = 0 
print 'start : ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
for i in gold_drug_id:
    other_drugs_sim = df_dr_sim[(df_dr_sim.Drug_id1 == i) & (df_dr_sim.Drug_id2 != i)] # 검사할 drug를 제외한 다른 drug들
    for j in gold_disease_id:
        target_disease = df_asso[(df_asso.disease_id ==int(j))] # 검사할 disease
        flags = other_drugs_sim['Drug_id2'].isin(target_disease['drug_id']) # 검사할 disease와 다른 drug들의 association여부
        sum_selected_drug = other_drugs_sim[flags]['Similarity'].sum() # association이 있는 것들의 합
        #sum_all_drug = other_drugs_sim['Similarity'].sum() # similarity의 전체 합
        #norm = sum_selected_drug / sum_all_drug #normalize한 값
        di_dr_mat[i].ix[j] = sum_selected_drug
    cnt = cnt + 1
    if cnt % 100 == 0:
        print 'cnt == {} : '.format(cnt) + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print 'end : ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S')

start : 2017-03-15 16:02:32
cnt == 100 : 2017-03-15 16:03:06
cnt == 200 : 2017-03-15 16:03:40
cnt == 300 : 2017-03-15 16:04:14
cnt == 400 : 2017-03-15 16:04:47
end : 2017-03-15 16:05:11


In [33]:
#2017.3.15. disease-drug distance #normalize 안한 버전
dr_di_mat = pd.DataFrame(columns = gold_drug_id, index= gold_disease_id) #dr에 대한 di의 distance보기
cnt = 0 
print 'start : ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
for i in gold_disease_id:
    other_diseases_sim = df_di_sim[(df_di_sim.Disease_id1 == int(i)) & (df_di_sim.Disease_id2 != int(i))] # 검사할 disease를 제외한 다른 disease들
    for j in gold_drug_id:
        target_drug = df_asso[(df_asso.drug_id ==j)] # 검사할 drug
        flags = other_diseases_sim['Disease_id2'].isin(target_drug['disease_id']) # 검사할 drug와 다른 disease들의 association여부
        sum_selected_disease = other_diseases_sim[flags]['Similarity'].sum() # association이 있는 것들의 합
        #sum_all_disease = other_diseases_sim['Similarity'].sum() # similarity의 전체 합
        #norm = sum_selected_disease / sum_all_disease #normalize한 값
        dr_di_mat[j].ix[i] = sum_selected_disease
    cnt = cnt + 1
    if cnt % 100 == 0:
        print 'cnt == {} : '.format(cnt) + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print 'end : ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S')

start : 2017-03-15 16:05:12
cnt == 100 : 2017-03-15 16:06:18
cnt == 200 : 2017-03-15 16:07:24
end : 2017-03-15 16:07:48


In [34]:
bef_distance = -(di_dr_mat+dr_di_mat)
distance_mat = pd.DataFrame(columns = bef_distance.columns.values, index = bef_distance.index.values)
cnt = 0 
print 'start : ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
for i in bef_distance.columns.values:
    for j in bef_distance.index.values:
        a = bef_distance[i].ix[j]
        distance = np.exp(a)
        distance_mat[i].ix[j] = distance
    cnt = cnt + 1
    if cnt % 100 == 0:
        print 'cnt == {} : '.format(cnt) + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print 'end : ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S')

start : 2017-03-15 16:07:48
cnt == 100 : 2017-03-15 16:07:55
cnt == 200 : 2017-03-15 16:08:02
cnt == 300 : 2017-03-15 16:08:09
cnt == 400 : 2017-03-15 16:08:17
end : 2017-03-15 16:08:22


In [35]:
print di_dr_mat.min().min()
print di_dr_mat.max().max()
print dr_di_mat.min().min()
print dr_di_mat.max().max()
print (di_dr_mat + dr_di_mat).min().min()
print (di_dr_mat + dr_di_mat).max().max()
print bef_distance.min().min()
print bef_distance.max().max()
print np.exp(bef_distance.min().min())
print np.exp(bef_distance.max().max())

0.0
67.4206821873
0.0
18.7744958819
0.0
80.7863062172
-80.7863062172
0.0
8.22153663889e-36
1.0


In [36]:
distance_mat.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/7_new_association/12_1_drug_disease_distance_removed.tsv", sep='\t')
distance_mat.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/7_new_association/12_1_drug_disease_distance_removed.tsv", sep='\t')

In [ ]:
# TN 찾기 2017.3.15.

In [100]:
# 전체 distance matrix에서 거리가 0.1보다 큰 애들을 찾는다. (거리가 먼 애들)
li_tn = list()
for index, row in distance_mat.iterrows():
    filtered = row[row> 0.1]
    if not filtered.empty:
        li_tn.append([index, filtered.index.tolist()]) 
df_tn = pd.DataFrame()
for i in li_tn:
    for j in i[1]:
        temp_df = pd.DataFrame(data = [[i[0], j]], columns = ['disease_id', 'drug_id'])
        df_tn = df_tn.append(temp_df)

In [101]:
print len(df_tn)
print len(df_tn.drop_duplicates('disease_id'))
print len(df_tn.drop_duplicates('drug_id'))
df_tn.head(1)

9339
86
166


,disease_id,drug_id
0,102400,DB00659


In [102]:
# 구한 TN이 positive associaion과 겹치는게 있는지 체크
duplicated = pd.DataFrame()
for index, row in df_asso.iterrows():
    li_row = row.tolist()
    temp = df_tn[(df_tn.disease_id == li_row[0]) & (df_tn.drug_id == li_row[1])]
    if temp.empty:
        continue
    else:
        duplicated = duplicated.append(temp)

In [103]:
# duplicated 를 제외한 나머지 dataframe 
duplicated['key'] = 0
tt = pd.merge(left = df_tn, right = duplicated, how='left')
df_tn2 = tt[tt.key != 0][['disease_id', 'drug_id']]

In [106]:
df_tn2.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/7_new_association/12_1_2_true_negative_whole_removed.tsv", sep='\t', index=False)
df_tn2.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/7_new_association/12_1_2_true_negative_whole_removed.tsv", sep='\t', index=False)

In [107]:
# di당 거리가 먼 dr을 일정비율로 뽑는다.
li_tn_di = list()
for index, row in distance_mat.iterrows():
    idx = int(len(row)*0.1)
    filtered = row.sort_values(ascending=False)[:idx]
    li_tn_di.append([index, filtered.index.tolist()])

df_tn_di = pd.DataFrame()
for i in li_tn_di:
    for j in i[1]:
        temp_df = pd.DataFrame(data=[[i[0], j]], columns = ['disease_id', 'drug_id'])
        df_tn_di= df_tn_di.append(temp_df)

In [108]:
print len(df_tn_di)
print len(df_tn_di.drop_duplicates('disease_id'))
print len(df_tn_di.drop_duplicates('drug_id'))
print len(df_tn_di.drop_duplicates())
df_tn_di.head(2)

11045
235
252
11045


,disease_id,drug_id
0,102300,DB00454
0,102300,DB01006


In [109]:
# 구한 TN이 positive associaion과 겹치는게 있는지 체크
duplicated = pd.DataFrame()
for index, row in df_asso.iterrows():
    li_row = row.tolist()
    temp = df_tn_di[(df_tn_di.disease_id == li_row[0]) & (df_tn_di.drug_id == li_row[1])]
    if temp.empty:
        continue
    else:
        duplicated = duplicated.append(temp)

In [110]:
print len(duplicated)

321


In [111]:
# duplicated 를 제외한 나머지 dataframe 
duplicated['key'] = 0
tt = pd.merge(left = df_tn_di, right = duplicated, how='left')
df_tn_di2 = tt[tt.key != 0][['disease_id', 'drug_id']]

In [113]:
df_tn_di2.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/7_new_association/12_2_3_true_negative_di_removed.tsv", sep='\t', index=False)
df_tn_di2.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/7_new_association/12_2_3_true_negative_di_removed.tsv", sep='\t', index=False)

In [115]:
# dr당 거리가 먼 di을 일정비율로 뽑는다.
li_tn_dr = list()
for column in distance_mat:
    idx = int(len(distance_mat[column])*0.1)
    filtered = distance_mat[column].sort_values(ascending=False)[:idx]
    li_tn_dr.append([column, filtered.index.tolist()])

df_tn_dr = pd.DataFrame()
for i in li_tn_dr:
    for j in i[1]:
        temp_df = pd.DataFrame(data=[[j, i[0]]], columns = ['disease_id', 'drug_id'])
        df_tn_dr= df_tn_dr.append(temp_df)

In [116]:
print len(df_tn_dr)
print len(df_tn_dr.drop_duplicates('disease_id'))
print len(df_tn_dr.drop_duplicates('drug_id'))
df_tn_dr.head()

10833
114
471


,disease_id,drug_id
0,145200,DB00190
0,182000,DB00190
0,246300,DB00190
0,208085,DB00190
0,137800,DB00190


In [117]:
# 구한 TN이 positive associaion과 겹치는게 있는지 체크
duplicated = pd.DataFrame()
for index, row in df_asso.iterrows():
    li_row = row.tolist()
    temp = df_tn_dr[(df_tn_dr.disease_id == li_row[0]) & (df_tn_dr.drug_id == li_row[1])]
    if temp.empty:
        continue
    else:
        duplicated = duplicated.append(temp)

In [118]:
len(duplicated)

182

In [119]:
# duplicated 를 제외한 나머지 dataframe 
duplicated['key'] = 0
tt = pd.merge(left = df_tn_dr, right = duplicated, how='left')
df_tn_dr2 = tt[tt.key != 0][['disease_id', 'drug_id']]

In [121]:
df_tn_dr2.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/7_new_association/12_2_4_true_negative_dr_removed.tsv", sep='\t', index=False)
df_tn_dr2.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/7_new_association/12_2_4_true_negative_dr_removed.tsv", sep='\t', index=False)

In [122]:
desc.head(1)

,drug_id,disease_id,association,C0008301,C0011603,C0015230,C0016382,C0018681,C0018965,C0020517,...,387585,388165,55219,55973,65264,80350,84624,84722,91851,406216
0,DB00190,102300,1,0,1,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [123]:
# negative descriptor 만들기 -whole
df_tn_desc = pd.DataFrame()
print 'start : ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
for index, row in df_tn2.iterrows():
    temp_df = desc[(desc.drug_id == row['drug_id']) & (desc.disease_id == int(row['disease_id']))]
    if temp_df.empty:
        continue
    else:
        df_tn_desc = df_tn_desc.append(temp_df)
print 'end : ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S')

start : 2017-03-15 19:58:48
end : 2017-03-15 20:16:13


In [124]:
desc_pos = desc[desc.association ==1]
frames = [desc_pos, df_tn_desc]
whole_desc = pd.concat(frames)

In [125]:
print len(whole_desc)
print len(whole_desc.drop_duplicates())

10727
10727


In [126]:
whole_desc.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/8_new_training/7_2_1_tn_whole_descriptor_removed.tsv", sep='\t', index=False)
whole_desc.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/7_2_1_tn_whole_descriptor_removed.tsv", sep='\t', index=False)

In [127]:
# negative descriptor 만들기 -disease
df_tn_di_desc = pd.DataFrame()
print 'start : ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
for index, row in df_tn_di2.iterrows():
    temp_df = desc[(desc.drug_id == row['drug_id']) & (desc.disease_id == int(row['disease_id']))]
    if temp_df.empty:
        continue
    else:
        df_tn_di_desc = df_tn_di_desc.append(temp_df)
print 'end : ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S')

start : 2017-03-15 20:17:21
end : 2017-03-15 20:26:55


In [128]:
#desc_pos = desc[desc.association ==1]
frames = [desc_pos, df_tn_di_desc]
di_desc = pd.concat(frames)

In [129]:
di_desc.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/8_new_training/7_2_2_tn_di_descriptor_removed.tsv", sep='\t', index=False)
di_desc.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/7_2_2_tn_di_descriptor_removed.tsv", sep='\t', index=False)

In [130]:
# negative descriptor 만들기 -drug
df_tn_dr_desc = pd.DataFrame()
print 'start : ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
for index, row in df_tn_dr2.iterrows():
    temp_df = desc[(desc.drug_id == row['drug_id']) & (desc.disease_id == int(row['disease_id']))]
    if temp_df.empty:
        continue
    else:
        df_tn_dr_desc = df_tn_dr_desc.append(temp_df)
print 'end : ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S')

start : 2017-03-15 20:27:25
end : 2017-03-15 20:36:01


In [131]:
print len(df_tn_dr_desc)
print len(df_tn_dr_desc.drop_duplicates('disease_id'))
print len(df_tn_dr_desc.drop_duplicates('drug_id'))
df_tn_dr_desc.head(1)

10651
56
471


,drug_id,disease_id,association,C0008301,C0011603,C0015230,C0016382,C0018681,C0018965,C0020517,...,387585,388165,55219,55973,65264,80350,84624,84722,91851,406216
54,DB00190,145200,0,0,1,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [132]:
print len(df_tn_dr)
print len(df_tn_dr.drop_duplicates('disease_id'))
print len(df_tn_dr.drop_duplicates('drug_id'))
df_tn_dr.head(1)

10833
114
471


,disease_id,drug_id
0,145200,DB00190


In [133]:
#desc_pos = desc[desc.association ==1]
frames = [desc_pos, df_tn_dr_desc]
dr_desc = pd.concat(frames)

In [134]:
dr_desc.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/8_new_training/7_2_3_tn_dr_descriptor_removed.tsv", sep='\t', index=False)
dr_desc.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/7_2_3_tn_dr_descriptor_removed.tsv", sep='\t', index=False)